<h2>Install boto3</h2>

pip install boto3

Set up credentials:
* create folder in /home/user/ named ".aws" (/home/user/.aws). This folder is hidden
* create file in folder .aws with name "credentials". Paste following lines. YOUR_KEY and YOUR_SECRET (http://docs.aws.amazon.com/general/latest/gr/getting-aws-sec-creds.html)

    [default] <br/>
    aws_access_key_id = YOUR_KEY <br/>
    aws_secret_access_key = YOUR_SECRET <br/>


* create file config and paste following lines

    [default] <br/>
    region=us-east-1 <br/>
    <br/>
* Check your accoutn id
https://docs.aws.amazon.com/IAM/latest/UserGuide/console_account-alias.html


In [ ]:
import boto3

import boto3
ec2 = boto3.resource('ec2')

instances = ec2.instances.filter(
    Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])
instanceListNames = []
attachedVolumes = []
for instance in instances:
    print(instance.id, instance.instance_type, instance.key_name, instance.private_ip_address, instance.public_dns_name ,instance.root_device_name)
    instanceListNames.append(instance.id)
    volumeId = instance.block_device_mappings
    attachedVolumes.append(volumeId[0]['Ebs']['VolumeId'])
    print(volumeId)

    

<h3> Create budget</h3>

In [ ]:
import datetime

budget = boto3.client('budgets')

create_budget_output = budget.create_budget(
    AccountId='xxxxxxxxxxxxxxxxxxxx',
    Budget={
        'BudgetName': 'BudgetInvestigation',
        'BudgetLimit': {
            'Amount': '10',
            'Unit': 'USD'
        },
        'CostTypes': {
            'IncludeTax': True,
            'IncludeSubscription': True,
            'UseBlended': True
        },
        'TimeUnit': 'MONTHLY',
        'TimePeriod': {
            'Start': datetime.datetime(2018, 1, 1),
            'End': datetime.datetime(2019, 1, 1)
        },
        'CalculatedSpend': {
            'ActualSpend': {
                'Amount': '10',
                'Unit': 'USD'
            },
            'ForecastedSpend': {
                'Amount': '10',
                'Unit': 'USD'
            }
        },
        'BudgetType': 'COST'
    },
    NotificationsWithSubscribers=[
        {
            'Notification': {
                'NotificationType': 'ACTUAL',
                'ComparisonOperator': 'LESS_THAN',
                'Threshold': 90.0,
                'ThresholdType': 'PERCENTAGE'
            },
            'Subscribers': [
                {
                    'SubscriptionType': 'EMAIL',
                    'Address': 'xxxxxxxx@xxxxxxxx.com'
                },
            ]
        },
    ]
)

<h3>Check budget</h3>

In [ ]:
client = boto3.client('budgets')
response = client.describe_budget(
    AccountId='xxxxxxxxxxxxxxxx',
    BudgetName='BudgetInvestigation'
)

actualSpend = float(response['Budget']['CalculatedSpend']['ActualSpend']['Amount'])

print(actualSpend)

<h3> Create lambda function</h3>

Roles IAM should be created in order to execute lambda scripts

In lambda function remember to update accoutn id and uncomment specific lines <br />
Code below should be zipped (.zip)

In [ ]:
#https://aws.amazon.com/premiumsupport/knowledge-center/start-stop-lambda-cloudwatch/

import boto3

ec2 = boto3.resource('ec2')

def lambda_handler(event, context):
    # TODO implement
    
    instances = ec2.instances.filter(
        Filters=[{'Name': 'instance-state-name', 'Values': ['running']}])
    instanceListNames = []
    attachedVolumes = []
    for instance in instances:
        print(instance.id, instance.instance_type, instance.key_name, instance.private_ip_address, instance.public_dns_name ,instance.root_device_name)
        instanceListNames.append(instance.id)
        volumeId = instance.block_device_mappings
        attachedVolumes.append(volumeId[0]['Ebs']['VolumeId'])
        print(volumeId)

    # Check budget
    # Create a budget on AWS
    # https://docs.aws.amazon.com/awsaccountbilling/latest/aboutv2/budgets-create.html
    # 
    # Check your accoutn id
    # https://docs.aws.amazon.com/IAM/latest/UserGuide/console_account-alias.html
    # 

    client = boto3.client('budgets')
    response = client.describe_budget(
        AccountId='xxxxxxxxxxxxxxxxx',
        BudgetName='BudgetInvestigation'
    )

    actualSpend = float(response['Budget']['CalculatedSpend']['ActualSpend']['Amount'])

    print(actualSpend)


    # If limit is exceeded create a snapshot of EBS and skill all instances

    if(actualSpend > 0):
        print('create snapshot')
        print(attachedVolumes)
        '''
        for volumeId in attachedVolumes:
            ec2.create_snapshot(VolumeId=volumeId)
        '''
        print('kill all instances')
        print(instanceListNames)
        '''
        ec2.instances.filter(InstanceIds=instanceListNames).stop()
        ec2.instances.filter(InstanceIds=instanceListNames).terminate()
        '''

    return 'Instances terminated'

Run the following code to send zipped file to AWS

In [ ]:
lambdaFunction = boto3.client('lambda')

with open('/path/aws_monitor_instances_lambda.zip', 'rb') as content_file:
    content = content_file.read()
    
vals = {}
vals['FunctionName'] = 'aws_monitor_instances_lambda'
vals['Role'] = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxx'
vals['Handler'] = 'aws_monitor_instances_lambda.lambda_handler'
# vals['Code'] = {'S3Bucket': 'https://s3-ap-southeast-1.amazonaws.com/abcde/aws-lambda-image.zip'}
vals['Code'] = {'ZipFile': content}
vals['Publish'] = True
vals['Runtime'] = 'python3.6'
vals['Timeout'] = 30

create_function_output = lambdaFunction.create_function(**vals)
RuleArn = create_function_output['FunctionArn']

<h3>Create CloudWatch Event</h3>

In [ ]:
cloudwatchEvents = boto3.client('events')

put_role_output = cloudwatchEvents.put_rule(
    Name='checkBudget',
    ScheduleExpression='rate(1 minute)',
    State='ENABLED',
    Description='checkBudget'
)

In [ ]:
put_targets_output = cloudwatchEvents.put_targets(
    Rule='checkBudget',
    Targets=[
        {
            'Id': '1',
            'Arn': RuleArn,
            
          },
    ]
)


Additionaly you can create CloudWatch Alarm <br/>

<h3> Create alarm in the Cloud Watch</h3>

In [ ]:
cloudwatch = boto3.client('cloudwatch')

# Create alarm
cloudwatch.put_metric_alarm(
    AlarmName='Budget_Monitoring',
    ComparisonOperator='GreaterThanThreshold',
    MetricName='EstimatedCharges',
    Namespace='AWS/Billing',
    Statistic='Maximum',
    Threshold=0.3,
    ActionsEnabled=False,
    Period=6,
    AlarmDescription='Alarm when bidget is exceeded',
    EvaluationPeriods=1
)

# AlarmName, AlarmDescription, ActionsEnabled, OKActions, AlarmActions, 
# InsufficientDataActions, MetricName, Namespace, Statistic, 
# ExtendedStatistic, Dimensions, Period, Unit, EvaluationPeriods, 
# Threshold, ComparisonOperator, TreatMissingData, EvaluateLowSampleCountPercentile
# Dimensions='Name=Currency,Value=USD',
